In [1]:
import warnings
import os.path

from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling,calculateRolling,calculateLagging,createDateFeatures,splitTimeWindow,calculateSimilarRolling,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
from __future__ import absolute_import, division, print_function, unicode_literals

import os, csv

import matplotlib.pyplot as plt

    
warnings.filterwarnings("ignore", category=DeprecationWarning)
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
cate_list = [7371]
for cate in cate_list:
    if cate in [7371]:
        pred_date = pd.to_datetime('2018-05-08')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [11922]:
        pred_date = pd.to_datetime('2017-10-10')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [7052, 7054]:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
    else:
        pred_date = pd.to_datetime('2018-03-10')
        scenario['lookforwardPeriodDays'] = 8
scenario['forecastStartDate'] = pred_date

In [16]:
pd.options.display.max_columns = 100

In [2]:
! pwd

/home/ubuntu/sunjiadong/promotion_offline


In [3]:
p1_used_header=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                        'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                        'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                        'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                        'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                        'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                        'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                        'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']


p2_used_header = ['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
             'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
             'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
             'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
             'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']

In [4]:
p1 = pd.read_csv('diannaobangong/7371_p1.csv',sep='\t',names = p1_used_header)
p1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778136 entries, 0 to 778135
Data columns (total 50 columns):
productkey                           778136 non-null int64
promotionkey                         778136 non-null object
startdatetime                        778136 non-null object
enddatetime                          778136 non-null object
jdprice                              778136 non-null float64
syntheticgrossprice                  778136 non-null float64
promotiondesc                        778136 non-null object
promotiondesc_flag                   778136 non-null int64
promotiontype                        778136 non-null object
promotionsubtype                     778136 non-null object
areatypearray                        778136 non-null object
tokenflag                            778136 non-null object
directdiscount_discount              778136 non-null object
directdiscount_availabilitynumber    778136 non-null object
bundle_subtype1_threshold            778136 non-n

In [8]:
p1 = f01(p1)

/home/ubuntu/wusai/wspy2/local/lib/python2.7/site-packages/pandas/core/missing.py:42: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  mask = arr == x


convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert to Float columns:
bundle_subtype4_giveaway3
convert to Float columns:
bundle_

In [17]:
p1

,productkey,promotionkey,startdatetime,enddatetime,jdprice,syntheticgrossprice,promotiondesc,promotiondesc_flag,promotiontype,promotionsubtype,areatypearray,tokenflag,directdiscount_discount,directdiscount_availabilitynumber,bundle_subtype1_threshold,bundle_subtype1_giveaway,bundle_subtype4_threshold1,bundle_subtype4_giveaway1,bundle_subtype4_threshold2,bundle_subtype4_giveaway2,bundle_subtype4_threshold3,bundle_subtype4_giveaway3,bundle_subtype2_threshold,bundle_subtype2_giveaway,bundle_subtype2_maximumgiveaway,bundle_subtype15_thresholdnumber1,bundle_subtype15_giveawayrate1,bundle_subtype15_thresholdnumber2,bundle_subtype15_giveawayrate2,bundle_subtype15_thresholdnumber3,bundle_subtype15_giveawayrate3,bundle_subtype6_thresholdnumber,bundle_subtype6_freenumber,suit_maxvaluepool,suit_minvaluepool,suit_avgvaluepool,suit_discount,directdiscount_saleprice,bundle_subtype1_percent,bundle_subtype4_percent,bundle_subtype2_percent,bundle_subtype15_percent,bundle_subtype6_percent,suit_percent,allpercentdiscount,mainproductkey,hierarchylevel3key,createdate,statuscode,dt
0,6831871.0,223950674.0,2018-06-25 00:00:00,2018-07-01 00:00:00,6.5,6.5,得力学生文具2件9折,0,10.0,15.0,{u'0': 1},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.1,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,-1.000000,0.1,-1.0,-1.0,0.100000,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
1,6831871.0,223950674.0,2018-07-01 00:00:00,2018-07-02 00:00:00,6.5,6.5,得力学生文具2件9折,0,10.0,15.0,{u'0': 1},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.1,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,-1.000000,0.1,-1.0,-1.0,0.100000,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
2,6831871.0,223950674.0,2018-07-02 00:00:00,2018-07-08 23:59:59,6.5,6.0,得力学生文具2件9折,0,10.0,15.0,{u'0': 1},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.1,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,-1.000000,0.1,-1.0,-1.0,0.100000,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
3,6831871.0,217514256.0,2018-04-23 00:00:00,2018-05-06 23:59:59,6.5,6.5,得力每满119减20,0,10.0,2.0,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,119.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,0.168067,-1.0,-1.0,-1.0,0.168067,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
4,6831871.0,221947361.0,2018-06-07 23:59:59,2018-06-15 23:59:59,6.5,6.5,得力每满139减40,0,10.0,2.0,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,0.287770,-1.0,-1.0,-1.0,0.287770,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
5,6831871.0,221947361.0,2018-06-07 00:00:00,2018-06-07 23:59:59,6.5,6.5,得力每满139减40,0,10.0,2.0,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,139.0,40.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.0,-1.0,-1.0,0.287770,-1.0,-1.0,-1.0,0.287770,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
6,6831871.0,NaN,2018-06-24 23:59:59,2018-06-25 00:00:00,6.5,6.5,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
7,6831871.0,NaN,2018-06-07 23:59:59,2018-06-15 23:59:59,6.5,6.5,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
8,6831871.0,NaN,2018-07-08 23:59:59,2018-07-09 00:00:00,6.5,6.0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12
9,6831871.0,NaN,2018-06-15 23:59:59,2018-06-16 00:00:00,6.5,6.5,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6831871.0,7371.0,2018-08-12,3001.0,2018-08-12


In [18]:
len(p1)

778136

In [9]:
len(p1) - p1.count()

The history saving thread hit an unexpected error (OperationalError('database or disk is full',)).History will not be written to the database.


productkey                                0
promotionkey                         367355
startdatetime                             0
enddatetime                               0
jdprice                                   0
syntheticgrossprice                       0
promotiondesc                        367355
promotiondesc_flag                        0
promotiontype                        367355
promotionsubtype                     367355
areatypearray                        663327
tokenflag                            622767
directdiscount_discount              673909
directdiscount_availabilitynumber    673909
bundle_subtype1_threshold            777022
bundle_subtype1_giveaway             777022
bundle_subtype4_threshold1           750173
bundle_subtype4_giveaway1            750173
bundle_subtype4_threshold2           750173
bundle_subtype4_giveaway2            750173
bundle_subtype4_threshold3           750173
bundle_subtype4_giveaway3            750173
bundle_subtype2_threshold       

In [5]:
p2 = pd.read_csv('diannaobangong/7371_p2.csv',sep='\t', names=p2_used_header)
p2.info()

/home/ubuntu/wusai/wspy2/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1299906 entries, 0 to 1299905
Data columns (total 40 columns):
ProductKey                1299906 non-null int64
Date                      1299906 non-null object
HierarchyLevel3Key        1299906 non-null int64
PromotionCount            1299906 non-null int64
bundlecount               1299906 non-null int64
MaxDiscount               1299906 non-null float64
MinDiscount               1299906 non-null float64
AvgDiscount               1299906 non-null float64
MaxSyntheticDiscountA     1299906 non-null object
MinSyntheticDiscountA     1299906 non-null object
AvgSyntheticDiscountA     1299906 non-null object
MaxBundleDiscount         1299906 non-null float64
MinBundleDiscount         1299906 non-null float64
AvgBundleDiscount         1299906 non-null float64
MaxDirectDiscount         1299906 non-null float64
MinDirectDiscount         1299906 non-null float64
AvgDirectDiscount         1299906 non-null float64
MaxFreegiftDiscount       129990

In [ ]:
path = 'diannaobangong/'
suffix = '.csv'
item = 'p1'
for_what = ['train', 'predict']
features_to_keep = []
# cate_list = [7054, 7052, 870]
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_f01
for fw in for_what:
    for cate in cate_list:
        print ("output and save: %s_p1_%s" %(str(cate),fw))
        p1_path = os.path.join(path, str(cate)+'_'+item+suffix)
        p1_used_header=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                        'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                        'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                        'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                        'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                        'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                        'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                        'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']
        print('read data from '+p1_path+'....')
        period_promo_raw = pd.read_csv(p1_path,sep='\t',header=None)
        period_promo_raw.columns=p1_used_header
        period_promo_raw_clean = period_promo_raw
        train_pred_gate = fw   # 'train'
        if cate in [7371]:
            pred_date = pd.to_datetime('2018-05-08')
            scenario['lookforwardPeriodDays'] = 7
        if cate in [11922]:
            pred_date = pd.to_datetime('2017-10-10')
            scenario['lookforwardPeriodDays'] = 7
        if cate in [7052, 7054]:
            pred_date = pd.to_datetime('2018-02-01')
            scenario['lookforwardPeriodDays'] = 10
        else:
            pred_date = pd.to_datetime('2018-02-01')
            scenario['lookforwardPeriodDays'] = 10
        f01_output = ow_f01.generate_f01_promo(area_rdc_map, period_promo_raw_clean,scenario, train_pred_gate, ForecastStartDate=pred_date)
#         f01_output.set_index('Date', inplace=True)
#         if fw == 'train':
#             print('#---------clear period by check variance:......--------#####')
#             f01_output, features_to_keep,features_to_del = clearbycheckvariance(f01_output)
#             print('features to keep:', features_to_keep)
#             print('features to delete:',features_to_del)
#             scenario['selectedColumns']['features'] = list(set(scenario['selectedColumns']['features']) - set(features_to_del))
#             print('#---------clear by check variance complete------#####')
#         else:
#             f01_output = f01_output[features_to_keep]
        print('saving data to '+os.path.join(path, train_pred_gate+'_'+item+'_'+str(cate)+'2018_05_08_ws.csv'))
        f01_output.to_csv(os.path.join(path, train_pred_gate+'_'+item+'_'+str(cate)+'_2018_05_08_ws.csv'),index=False)
        print('-------saving complete---------')

output and save: 7371_p1_train
read data from diannaobangong/7371_p1.csv....
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert 

In [ ]:
ForecastStartDate = pd.to_datetime(scenario['forecastStartDate'])
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])
PredictEndDate = ForecastStartDate + datetime.timedelta(days=(scenario['lookforwardPeriodDays']-1))

In [6]:

#p1_path = 'tmp/0411_2018_brand/p1_0411.csv' 

path = 'tmp/data/'
suffix = '.da'
item = 'p1'
for_what = ['train', 'predict']
cate_list = [7052] #, 7052, 870]
# scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
# scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')


In [7]:
pd.options.display.max_columns = 1000

In [8]:
for fw in for_what:
    for cate in cate_list:
        print ("output and save: %s_p1_%s" %(str(cate),fw))
        p1_path = os.path.join(path, str(cate)+'_'+item+suffix)
        
        p1_used_header=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                        'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                        'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                        'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                        'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                        'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                        'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                        'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']
        print('read data from '+p1_path+'....')
        period_promo_raw = pd.read_csv(p1_path,sep='\t',header=None)
        period_promo_raw.columns=p1_used_header
        period_promo_raw_clean = period_promo_raw
        train_pred_gate = fw   # 'train'
        if cate in [11922]:
            pred_date = pd.to_datetime('2017-10-10')
            scenario['lookforwardPeriodDays'] = 7
        if cate in [7052, 7054]:
            pred_date = pd.to_datetime('2018-02-01')
            scenario['lookforwardPeriodDays'] = 10
        else:
            pred_date = pd.to_datetime('2018-02-01')
            scenario['lookforwardPeriodDays'] = 10
        f01_output = ow_f01.generate_f01_promo(area_rdc_map, period_promo_raw_clean,scenario, train_pred_gate, ForecastStartDate=pred_date)
        print('saving data to '+os.path.join(path, train_pred_gate+'_'+item+'_'+str(cate)+'_2018_02_01ws.csv'))
        f01_output.to_csv(os.path.join(path, train_pred_gate+'_'+item+'_'+str(cate)+'_2018_02_01ws.csv'),index=False)
        print('-------saving complete---------')

output and save: 7052_p1_train
read data from tmp/data/7052_p1.da....


KeyboardInterrupt: 

In [5]:
p1_raw.columns=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                        'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                        'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                        'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                        'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                        'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                        'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                        'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']

In [6]:
def dealwithnull(period_promo_raw):
    period_promo_raw.replace('null', np.nan, inplace=True)
    period_promo_raw.replace(-999, np.nan, inplace=True)
    period_promo_raw.replace(' null', np.nan, inplace=True)
    period_promo_raw.replace('None', np.nan, inplace=True)

In [7]:
def f01(period_promo_raw):
    dealwithnull(period_promo_raw)
    # to_float_col = ['productkey', 'promo_inperiod_flag', 'mainproductkey', 'hierarchylevel1key', 'hierarchylevel2key', 'hierarchylevel3key', 'statuscode', 'promotionkey', 'jdprice', 'syntheticgrossprice', 'promotiontype', 'promotionsubtype', 'levelmember', 'directdiscount_productkey', 'directdiscount_maxjdprice', 'directdiscount_minjdprice', 'directdiscount_avgjdprice', 'directdiscount_maxscrapedprice', 'directdiscount_minscrapedprice', 'directdiscount_avgscrapedprice', 'directdiscount_discount', 'directdiscount_price', 'directdiscount_limitationflag', 'directdiscount_limitationcode', 'directdiscount_availabilitynumberflag', 'directdiscount_availabilitynumber', 'directdiscount_minimumnumberflag', 'directdiscount_minimumnumber', 'directdiscount_maximumnumberflag', 'directdiscount_maximumnumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway', 'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3', 'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1', 'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3', 'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool', 'suit_discount', 'freegift_productkey', 'freegift_maxscrapedprice', 'freegift_minscrapedprice', 'freegift_avgscrapedprice', 'freegift_maxjdprice', 'freegift_minjdprice', 'freegift_avgjdprice', 'freegift_freegiftvalue', 'freegift_freegiftvalueatjdprice', 'freegift_freegiftvalueatscrapedprice', 'directdiscount_saleprice', 'directdiscount_percent', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent', 'bundle_subtype6_percent', 'suit_percent', 'freegift_percent', 'allpercentdiscount']
    to_float_col = ['allpercentdiscount','bundle_subtype15_giveawayrate1','bundle_subtype15_giveawayrate2','bundle_subtype15_giveawayrate3','bundle_subtype15_percent','bundle_subtype15_thresholdnumber1','bundle_subtype15_thresholdnumber2','bundle_subtype15_thresholdnumber3','bundle_subtype1_giveaway','bundle_subtype1_percent','bundle_subtype1_threshold','bundle_subtype2_giveaway','bundle_subtype2_maximumgiveaway','bundle_subtype2_percent','bundle_subtype2_threshold','bundle_subtype4_giveaway1','bundle_subtype4_giveaway2','bundle_subtype4_giveaway3','bundle_subtype4_percent','bundle_subtype4_threshold1','bundle_subtype4_threshold2','bundle_subtype4_threshold3','bundle_subtype6_freenumber','bundle_subtype6_percent','bundle_subtype6_thresholdnumber','directdiscount_availabilitynumber','directdiscount_discount','directdiscount_saleprice','hierarchylevel3key','jdprice','mainproductkey','productkey','promotionkey','promotionsubtype','promotiontype','statuscode','suit_avgvaluepool','suit_discount','suit_maxvaluepool','suit_minvaluepool','suit_percent','syntheticgrossprice']
    period_promo_raw['enddatetime'] = pd.to_datetime(period_promo_raw['enddatetime'])
    period_promo_raw['startdatetime'] = pd.to_datetime(period_promo_raw['startdatetime'])
    period_promo_raw['dt'] = pd.to_datetime(period_promo_raw['dt'])

    if 'object' in period_promo_raw[to_float_col].dtypes.values:
        object2Float(period_promo_raw,to_float_col)
    return period_promo_raw

In [113]:
def f01_p2(promoCalendarDf, DataStartDate, PredictEndDate, ForecastStartDate=None, train=True):
    if ForecastStartDate == None:
        ForecastStartDate = scenario['forecastStartDate']
    promoCalendarDf['Date'] = pd.to_datetime(promoCalendarDf['Date'])
    promoCalendarDf['dt'] = pd.to_datetime(promoCalendarDf['dt'])
    dealwithnull(promoCalendarDf)
    promoCalendarDf.drop_duplicates(inplace=True)
    
    # Convert Object -> float
    if 'object' in promoCalendarDf.dtypes.values:
        obj_cols = get_column_by_type(promoCalendarDf,'object')
        object2Float(promoCalendarDf,obj_cols)
    if train:
        promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']<ForecastStartDate)&(promoCalendarDf['Date']>DataStartDate)]
    else:
        promoCalendarDf = promoCalendarDf[(promoCalendarDf['Date']>=ForecastStartDate)&(promoCalendarDf['Date']<=PredictEndDate)]
    columns_list = promoCalendarDf.columns.tolist()
    if columns_list[0] == 'ProductKey' and columns_list[1] == 'Date':
        neworder = [columns_list[1]] + [columns_list[0]] + list(columns_list[2:])
        promoCalendarDf = promoCalendarDf[neworder]
    return promoCalendarDf

In [13]:
def checknullproportion(df):
    count = len(df) - df.count()
    return count/len(df)

In [77]:
def clearbycheckvariance(df):
    """
    clear zero variance columns
    The first columns is Date, Date type has no variance, so just keep it in df!
    """
    enum_df = df.select_dtypes(exclude=['int64','float64'])
    num_df = df.select_dtypes(include=['int64', 'float64'])
#     print(enum_df.columns.values, len(enum_df.columns.values))
#     print(num_df.columns.values, len(num_df.columns.values))
    var = num_df.var(axis=0)
    zero_var_features = var.index[var==0]
#     print(zero_var_features.values)
    to_keep = num_df.columns.difference(zero_var_features).values
    print(to_keep, len(to_keep))
    print ('number of zero variance columns: {}'.format(len(zero_var_features)))
#     print (df.shape)
    new_df = num_df[to_keep]
    num_df = num_df.loc[:, var != 0]
#     print('No.1',new_df.columns,len(new_df.columns))
#     print('No.2',num_df.columns,len(num_df.columns))
    if set(new_df.columns.values) == set(num_df.columns.values):
        print('#-__-Fine!')
    else:
        print('#- _ - Bad news!')
        return False
    df = pd.concat([enum_df, num_df],axis=1)
    to_keep = np.append(enum_df.columns.values, to_keep)
    print(to_keep,len(to_keep))
    print ('shape of new dataframe: ',df.shape)
    return df, to_keep

In [78]:
clearbycheckvariance(p1_df)

['ProductKey' 'bd_discount_daily_max' 'bd_discount_sgp_wgt'
 'bd_discount_wgt' 'bd_price_daily_min' 'bd_price_weighted'
 'bundle_buy199get100_ind' 'bundle_ind' 'cnt_period'
 'dd_discount_daily_max' 'dd_discount_sgp_daily_max' 'dd_discount_sgp_wgt'
 'dd_discount_wgt' 'dd_ind' 'dd_price_daily_min' 'dd_price_weighted'
 'flashsale_ind' 'jdprice_daily_min' 'sgp_daily_min'] 19
number of zero variance columns: 0
#-__-Fine!
['Date' 'ProductKey' 'bd_discount_daily_max' 'bd_discount_sgp_wgt'
 'bd_discount_wgt' 'bd_price_daily_min' 'bd_price_weighted'
 'bundle_buy199get100_ind' 'bundle_ind' 'cnt_period'
 'dd_discount_daily_max' 'dd_discount_sgp_daily_max' 'dd_discount_sgp_wgt'
 'dd_discount_wgt' 'dd_ind' 'dd_price_daily_min' 'dd_price_weighted'
 'flashsale_ind' 'jdprice_daily_min' 'sgp_daily_min'] 20
shape of new dataframe:  (1170125, 20)


(               Date  ProductKey  dd_price_weighted  bd_price_weighted  \
 0        2014-10-01    244923.0             178.00             178.00   
 1        2014-10-01   1191830.0             718.00             718.00   
 2        2014-10-01   1231000.0              88.00              88.00   
 3        2014-10-01   1231001.0              88.00              88.00   
 4        2014-10-01   1434881.0             238.00             238.00   
 5        2014-10-01   1434882.0             238.00             238.00   
 6        2014-10-01   1434904.0              65.00              65.00   
 7        2014-10-01   1434905.0              65.00              65.00   
 8        2014-10-01   1661588.0            1686.72            1686.72   
 9        2014-10-01   1970070.0              69.00              69.00   
 10       2014-10-01   2108765.0             179.00             179.00   
 11       2014-10-01   2108785.0             179.00             179.00   
 12       2014-10-01   2108790.0      

把空值比例过大（这个标准该如何界定）的特征去掉，根据特征意义有些也需要去掉

# Feature Selection beta

In [12]:
# -*- coding: utf-8 -*-
def feature_select(data, Ycol='sales', k=10, project_path='tmp/data/'):
    from sklearn.ensemble import RandomForestClassifier
    
    enum_df = df.select_dtypes(exclude=['int64','float64'])
    num_df = df.select_dtypes(include=['int64', 'float64'])
    print(enum_df.columns)
    print(num_df.columns)
    col_row = data.shape
    row = col_row[0]
    col = col_row[1]
    print(data,end='\n\n')
    X= data.drop(Ycol,axis=1)
    print(X,end='\n\n')
    Y= data[Ycol]
    Y.values.reshape(len(Y),-1)

    Z = pd.concat([X,Y], axis = 1)
    y=Y.ravel()

    forest = RandomForestClassifier()
    forest.fit(X.values, y)

    importances = forest.feature_importances_
    std = np.std([tree.feature_importances_ for tree in forest.estimators_],axis=0)
    indices = np.argsort(importances)[::-1]
    cut_indices = indices[0:k]

    header_indices = np.copy(cut_indices)
    topk_indices = np.append(header_indices,[-1])


    out = data.iloc[:,topk_indices]
    out.to_csv(project_path + "RF_top{}.csv".format(k),index=False)

    # Print the feature ranking
    print("Feature ranking:")
    for count in range(k):
        print("feature name:{}".format(Z.columns[header_indices[k]]))
        print("%d. no.%d (%f)" % (count+1, indices[count], importances[indices[count]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(k), importances[cut_indices],color="b", yerr=std[cut_indices], align="center")
    plt.xticks(range(k), cut_indices)
    plt.xlim([-1, k])
    plt.show()
    return out

In [14]:
ts_df_pred

,Date,ind,RDCKey,ProductKey,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,brand_code,sales,priceAfterDiscount,jd_prc,vendibility,counterState,salesForecast,reserveState,stockQuantity,utc_flag
83,2018-01-31,83,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
84,2018-01-30,84,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
85,2018-01-29,85,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
86,2018-01-28,86,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
87,2018-01-27,87,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
88,2018-01-26,88,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
89,2018-01-25,89,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
90,2018-01-24,90,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
91,2018-01-23,91,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0
92,2018-01-22,92,3.0,1317085.0,1319.0,1523.0,7052.0,8241.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
feature_select(ts_df_pred)

               Date  ind  RDCKey  ProductKey  HierarchyLevel1Key  \
83       2018-01-31   83     3.0   1317085.0              1319.0   
84       2018-01-30   84     3.0   1317085.0              1319.0   
85       2018-01-29   85     3.0   1317085.0              1319.0   
86       2018-01-28   86     3.0   1317085.0              1319.0   
87       2018-01-27   87     3.0   1317085.0              1319.0   
88       2018-01-26   88     3.0   1317085.0              1319.0   
89       2018-01-25   89     3.0   1317085.0              1319.0   
90       2018-01-24   90     3.0   1317085.0              1319.0   
91       2018-01-23   91     3.0   1317085.0              1319.0   
92       2018-01-22   92     3.0   1317085.0              1319.0   
93       2018-01-21   93     3.0   1317085.0              1319.0   
94       2018-01-20   94     3.0   1317085.0              1319.0   
95       2018-01-19   95     3.0   1317085.0              1319.0   
96       2018-01-18   96     3.0   1317085.0    

TypeError: float() argument must be a string or a number

# train

In [19]:
ts_df_pred.var(axis=0)

NameError: name 'ts_df_pred' is not defined

In [22]:
(p1_df.var(axis = 0) == 0).

ProductKey                   False
dd_price_weighted            False
bd_price_weighted            False
flashsale_ind                False
dd_ind                       False
bundle_ind                   False
bundle_buy199get100_ind      False
cnt_period                   False
dd_discount_daily_max        False
dd_discount_sgp_daily_max    False
bd_discount_daily_max        False
dd_price_daily_min           False
bd_price_daily_min           False
jdprice_daily_min            False
sgp_daily_min                False
dd_discount_wgt              False
dd_discount_sgp_wgt          False
bd_discount_wgt              False
bd_discount_sgp_wgt          False
dtype: bool

In [4]:
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling,calculateRolling,calculateLagging,createDateFeatures,splitTimeWindow,calculateSimilarRolling,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
warnings.filterwarnings("ignore", category=DeprecationWarning)

path = 'tmp/data/'
result_path = 'tmp/data/wusai/'
cate_list = [7054]#[870]
#cate_list = [7052, 7054, 870]
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
'''online analysis'''
#scenario['use_cart_feature'] = False
#scenario['use_promo_features'] = False
holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_train
for cate in cate_list:
    if cate in [11922]:
        pred_date = pd.to_datetime('2017-10-10')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [7052, 7054]:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
    else:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
        #pred_date = pd.to_datetime('2018-03-10')
        #scenario['lookforwardPeriodDays'] = 8

    pro_canlender_path = os.path.join(path, str(cate)+'_'+'p2.da')
    promoCalendarDf = pd.read_csv(pro_canlender_path,sep='\t',header=None)
    promoCalendarDf.columns=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
             'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
             'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
             'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
             'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']

    p1_out_path = os.path.join(path, 'train'+'_p1_'+str(cate)+'_2018_02_01ws.csv')
    period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

#     sea_path = os.path.join(path, str(cate)+'_season.csv')
    sea_path = os.path.join(path, '870_season.csv')
    seasonality_df = pd.read_csv(sea_path, parse_dates=['Date'])

    file_path = os.path.join(path, str(cate)+'_'+'ts.da')
    ts_df = pd.read_csv(file_path,header=None,sep='\t')
    model,feature=ow_train.train(area_rdc_map,period_promo_raw,promoCalendarDf,ts_df,scenario,holidays_df,seasonality_df,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)
    #feature.to_csv(os.path.join(result_path, str(cate)+'_train_feature_with_season_180201_0515.csv'),index=False)
    save_object(model, os.path.join(result_path, str(cate)+'_train_model_with_season_180201_0515'+'.pkl'))

convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag


ow_train.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace('null', np.nan, inplace=True)
ow_train.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace(-999, np.nan, inplace=True)
code/refactor/common.py:2178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype('float')


convert to Float columns:
RDCKey
convert to Float columns:
ProductKey
convert to Float columns:
HierarchyLevel1Key
convert to Float columns:
HierarchyLevel2Key
convert to Float columns:
HierarchyLevel3Key
convert to Float columns:
brand_code
convert to Float columns:
sales
convert to Float columns:
priceAfterDiscount
convert to Float columns:
jd_prc
convert to Float columns:
vendibility
convert to Float columns:
counterState
convert to Float columns:
salesForecast
convert to Float columns:
reserveState
convert to Float columns:
stockQuantity
convert to Float columns:
utc_flag
use cart feature?
False
| Add date features...
| - Add month of the year...
| - Add day of the week...
| Calculating national rolling features...
('| - Rolling value:', 'xxxHashColumn')
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 28 mean...
| - Rolling decay 14

| - Rolling decay 14 mean...
| - Rolling decay 7 mean...
| - Rolling decay 3 mean...
| - Added rolling features in 0.930106 seconds 

| - Added rolling features in 25.386726 seconds 

| Calculating volume rolling features...
('| - Rolling value:', 'salesForecast')
| - Shift 365...
| - Shift 180...
| - Shift 90...
| - Shift 30...
| - Shift 7...
| - Added rolling features in 1.592997 seconds 

featuresDf shape
(213245, 141)
rdc
3.0
rdc
4.0
rdc
5.0
rdc
6.0
rdc
9.0
rdc
10.0
rdc
316.0
rdc
772.0


In [7]:
feature.columns.str.contains('discount')

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False,  True, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

# predict

In [32]:
# -*- coding: utf-8 -*-
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling_predict,calculateRolling_predict,calculateLagging_predict,createDateFeatures,splitTimeWindow,calculateSimilarRolling_predict,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
import pickle

scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)

#scenario['lookforwardPeriodDays'] = 1
#scenario['use_cart_feature'] = False
#scenario['use_promo_features'] = False
path = 'tmp/data/'
result_path = 'tmp/data/wusai/'
cate_list = [7054]#[7054]
#cate_list = [7052, 7054, 870]

holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_predict
#cate = 7052
for cate in cate_list:
    pro_canlender_path = os.path.join(path, str(cate)+'_'+'p2.da')
    promoCalendarDf = pd.read_csv(pro_canlender_path,sep='\t',header=None)
    promoCalendarDf.columns=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
           'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
           'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
           'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
           'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']

    #p1_out_path = os.path.join(path, 'predict'+'_p1_'+str(cate)+'_2018_02_01.csv')
    p1_out_path = os.path.join(path, 'predict'+'_p1_'+str(cate)+'.csv')
    period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

    file_path = file_path = os.path.join(path, str(cate)+'_'+'ts.da')
    ts_df = pd.read_csv(file_path,header=None,sep='\t')

    model_path = os.path.join(result_path, str(cate)+'_train_model'+'_with_season_180201_0515'+'.pkl')
    with open(model_path,'r') as input:
        model = pickle.load(input)

    sea_path = os.path.join(path, '870_season.csv')
    seasonality_df = pd.read_csv(sea_path, parse_dates=['Date'])

    if cate in [11922]:
        pred_date = pd.to_datetime('2017-10-10')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [7052, 7054]:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
    else:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
        #pred_date = pd.to_datetime('2018-03-10')
        #scenario['lookforwardPeriodDays'] = 30
    result,df_fut=ow_predict.predict(area_rdc_map,period_promo_raw,promoCalendarDf,ts_df,scenario,holidays_df,model,seasonality_df,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)
    #result.to_csv('tmp/data/0425_result/'+'result_'+str(cate)+'.csv',index=False)
    result.to_csv(result_path+'result_'+str(cate)+'_zero_promo_0515.csv',index=False)


ow_predict.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace('null', np.nan, inplace=True)
ow_predict.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace(-999, np.nan, inplace=True)
ow_predict.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df.replace('None', np.nan, inplace=True)


convert to Float columns:
RDCKey
convert to Float columns:
ProductKey
convert to Float columns:
HierarchyLevel1Key
convert to Float columns:
HierarchyLevel2Key
convert to Float columns:
HierarchyLevel3Key
convert to Float columns:
brand_code
convert to Float columns:
sales
convert to Float columns:
priceAfterDiscount
convert to Float columns:
jd_prc
convert to Float columns:
vendibility
convert to Float columns:
counterState
convert to Float columns:
salesForecast
convert to Float columns:
reserveState
convert to Float columns:
stockQuantity
convert to Float columns:
utc_flag
convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDisco

code/refactor/common.py:1363: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  + df_past[similarLevels[2]]
code/refactor/common.py:1364: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_past['SyntheticPromotionSubType'] = df_past['SyntheticPromotionSubType'].fillna(0.0)


| - group by SyntheticPromotionSubType...


ow_predict.py:454: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred_df['ypred'] =ypred
ow_predict.py:455: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  pred_df['RDCKey'] = rdc
/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.

## simplifying the ts dataframe

In [90]:
cate_list

[7054]

In [18]:
scenario

{'After_price_col': 'priceAfterDiscount',
 'Before_price_col': 'red_price',
 'Date_col': 'Date',
 'Floor2zero_threshold': 0.4,
 'Fut_use_his_days': 90,
 'Min_instance': 300,
 'Min_onstock_day': 28,
 'Min_sku': 30,
 'N_days': 10,
 'Pred_length': 91,
 'RDCKeys': [3, 4, 5, 6, 9, 10, 316, 772],
 'Remove_by_stat_date': ['2015-11-01',
  '2015-11-02',
  '2015-11-11',
  '2015-11-12',
  '2015-12-12',
  '2016-06-05',
  '2016-06-06',
  '2016-06-18',
  '2016-06-19',
  '2016-06-20',
  '2016-06-17',
  '2016-06-16',
  '2016-11-10',
  '2016-11-11',
  '2016-11-12',
  '2016-12-12',
  '2017-06-18',
  '2017-06-19',
  '2017-06-20',
  '2017-06-17',
  '2017-06-16',
  '2017-11-10',
  '2017-11-11',
  '2017-11-12',
  '2017-12-12',
  '2018-06-18',
  '2018-06-19',
  '2018-06-20',
  '2018-06-17',
  '2018-06-16',
  '2018-11-10',
  '2018-11-11',
  '2018-11-12',
  '2018-12-12'],
 'STOCK_PRICE_FLAG': False,
 'Sku_col': 'ProductKey',
 'Target_col': 'salesForecast',
 'analysisPeriods': ['2017-02-27', '2017-04-24', '2017

In [92]:
ts_df = pd.read_csv(file_path,header=None,sep='\t')
ts_df.columns = ['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
ts_df['Date'] = pd.to_datetime(ts_df['Date'])
df = ts_df[ts_df.Date.between('2018-01-01','2018-04-01')]

In [99]:
condition= (ts_df['HierarchyLevel3Key']==7054) &  (ts_df['Date'] <= '2018-02-10') & (ts_df['Date'] >= '2018-02-01')&(ts_df.ProductKey==255778)

# analysis

In [46]:
! ls ./tmp/11922_2018/

at180323_11922_df_fut_using_mean_20171010.csv
at180323_11922_v_oct_feature_20171010.csv
at180328_11922_use_cart_feature_20171010.csv
model_no_promo_ow_20171010_rdc_at180321_11922.pkl
model_no_promo_ow_20171101_rdc_at180321_11922.pkl
model_no_promo_ow_20171201_rdc_at180322_11922.pkl
model_nousing_cart_20171010_rdc_at180329_11922.pkl
model_nousing_cart_step_1_20171010_rdc_at180329_11922.pkl
model_ow_20171101_rdc_at180321_11922.pkl
model_ow_20171201_rdc_at180322_11922.pkl
model_ow_v_oct_20171010_rdc_at180321_11922.pkl
model_ow_v_oct_20171010_rdc_at180323_11922.pkl
model_sku_188059_test_0411.pkl
model_use_cart_20171010_rdc_at180328_11922.pkl
model_using_cart_20171010_rdc_at180329_11922.pkl
model_using_cart_step_1_20171010_rdc_at180329_11922.pkl
model_using_national_cart_step_1_20171010_rdc_at180329_11922.pkl
p1_out_2018_predict.csv
plot
result_11922_171010_180321.csv
result_11922_171101_180321.csv
result_11922_171201_180322.csv
result_no_promo_11922_171010_180321.csv
result_no_promo_11922_

In [119]:
###anlysis###
import pandas as pd
import numpy as np
df1 = pd.read_csv(result_path+'result_'+str(cate)+'_zero_promo_0515.csv') #target ?? where is target？
df1['Date'] = pd.to_datetime(df1['Date'])
# df2 = pd.read_csv('tmp/11922_2018/result_no_promo_11922_171201_180322.csv') #target
cols = ['Date','HierarchyLevel3Key','ProductKey','RDCKey','ypred']
df11 = df1[cols]
print df11
# df22 = df2[cols]

# file_path = '/home/ubuntu/yulong/promotion_offline/tmp/11922_2018/11922_predict_ts'
# ts1_df = pd.read_csv(file_path,header=None,sep='\t')
# ts1_df.columns = ['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
#ts1_df['Date'] = pd.to_datetime(ts1_df['Date'])
groupkeys = ['Date','HierarchyLevel3Key','ProductKey','RDCKey']

df11 = pd.merge(df11[groupkeys+['ypred']],ts_df[groupkeys+['salesForecast']],how='left',on=groupkeys)
# df22 = pd.merge(df22[groupkeys+['ypred']],ts_df[groupkeys+['salesForecast']],how='left',on=groupkeys)

df111 = df11.groupby(['HierarchyLevel3Key','ProductKey','RDCKey']).sum().reset_index()
# df222 = df22.groupby(['HierarchyLevel3Key','ProductKey','RDCKey']).sum().reset_index()
df111.RDCKey.drop_duplicates()
print "###promo###"
for rdc in list(df111.RDCKey.unique()):
#for rdc in [3,4,5,6,9,10,316,772]:
    this_df = df111[df111.RDCKey==rdc]
    this_df['resi'] = np.abs(this_df['salesForecast']-this_df['ypred'])
    print "rdc:" + str(rdc)
    print this_df['resi'].sum()/this_df['salesForecast'].sum()
# print "###no_promo###"
# for rdc in list(df222.RDCKey.unique()):
# #for rdc in [3,4,5,6,9,10,316,772]:
#     this_df = df222[df222.RDCKey==rdc]
#     this_df['resi'] = np.abs(this_df['salesForecast']-this_df['ypred'])
#     print "rdc:" + str(rdc)
#     print this_df['resi'].sum()/this_df['salesForecast'].sum()

           Date  HierarchyLevel3Key  ProductKey  RDCKey      ypred
0    2018-02-01              7054.0    255778.0     3.0  24.334154
1    2018-02-02              7054.0    255778.0     3.0  24.064210
2    2018-02-03              7054.0    255778.0     3.0  24.064210
3    2018-02-04              7054.0    255778.0     3.0  24.064210
4    2018-02-05              7054.0    255778.0     3.0  23.172570
5    2018-02-06              7054.0    255778.0     3.0  24.448800
6    2018-02-07              7054.0    255778.0     3.0  24.076155
7    2018-02-08              7054.0    255778.0     3.0  23.740170
8    2018-02-09              7054.0    255778.0     3.0  24.010431
9    2018-02-10              7054.0    255778.0     3.0  35.288452
10   2018-02-01              7054.0    255780.0     3.0  16.638287
11   2018-02-02              7054.0    255780.0     3.0  17.832298
12   2018-02-03              7054.0    255780.0     3.0   8.122601
13   2018-02-04              7054.0    255780.0     3.0   6.70

/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


rdc:5.0
0.36879849108242035
rdc:6.0
0.3108316392612193
rdc:9.0
0.40946738210782396
rdc:10.0
0.3759576636740415
rdc:316.0
0.40072625786024835
rdc:772.0
0.30556349692456247


In [116]:
condition= (ts_df['HierarchyLevel3Key']==7054) &  (ts_df['Date'] <= '2018-02-10') & (ts_df['Date'] >= '2018-02-01')
ts_df[condition&(ts_df['ProductKey']==255778)]

,Date,ind,RDCKey,ProductKey,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,brand_code,sales,priceAfterDiscount,jd_prc,vendibility,counterState,salesForecast,reserveState,stockQuantity,utc_flag
105761,2018-02-10,73,3,255778,1319,1523,7054,14633,21.0,149.0,168.0,1.0,1.0,21.0,0.0,469.0,0
105762,2018-02-09,74,3,255778,1319,1523,7054,14633,8.0,157.63,164.25,1.0,1.0,8.0,0.0,478.0,0
105763,2018-02-08,75,3,255778,1319,1523,7054,14633,22.0,149.0,168.0,1.0,1.0,22.0,0.0,482.0,0
105764,2018-02-07,76,3,255778,1319,1523,7054,14633,19.0,157.42,168.0,1.0,1.0,19.0,0.0,344.0,0
105765,2018-02-06,77,3,255778,1319,1523,7054,14633,26.0,149.62,168.0,1.0,1.0,26.0,0.0,425.0,0
105766,2018-02-05,78,3,255778,1319,1523,7054,14633,21.0,158.67,166.86,1.0,1.0,21.0,0.0,437.0,0
105767,2018-02-04,79,3,255778,1319,1523,7054,14633,21.0,149.0,168.0,1.0,1.0,21.0,0.0,444.0,0
105768,2018-02-03,80,3,255778,1319,1523,7054,14633,27.0,149.0,168.0,1.0,1.0,27.0,0.0,258.0,0
105769,2018-02-02,81,3,255778,1319,1523,7054,14633,29.0,149.0,168.0,1.0,1.0,29.0,0.0,51.0,0
105770,2018-02-01,82,3,255778,1319,1523,7054,14633,26.0,148.81,167.08,1.0,1.0,26.0,0.0,87.0,0


In [82]:
df11['ypred'].sum()

20388.8211019408

In [81]:
ts_df.loc[condition]['salesForecast'].sum()

21553.0

In [51]:
ts1_df

,Date,ind,RDCKey,ProductKey,HierarchyLevel1Key,HierarchyLevel2Key,HierarchyLevel3Key,brand_code,sales,priceAfterDiscount,jd_prc,vendibility,counterState,salesForecast,reserveState,stockQuantity,utc_flag
0,2018-01-01,0,3,1220900,1316,1386,11922,15799,2.0,78.50,85.00,1.0,1.0,2.0,0.0,40.0,0
1,2017-12-31,1,3,1220900,1316,1386,11922,15799,1.0,72.00,85.00,1.0,1.0,1.0,0.0,41.0,0
2,2017-12-30,2,3,1220900,1316,1386,11922,15799,1.0,72.00,85.00,1.0,1.0,1.0,0.0,42.0,0
3,2017-12-29,3,3,1220900,1316,1386,11922,15799,0.0,NaN,NaN,1.0,1.0,0.0,0.0,43.0,0
4,2017-12-28,4,3,1220900,1316,1386,11922,15799,0.0,NaN,NaN,1.0,1.0,0.0,0.0,43.0,0
5,2017-12-27,5,3,1220900,1316,1386,11922,15799,2.0,76.60,80.60,1.0,1.0,2.0,0.0,43.0,0
6,2017-12-26,6,3,1220900,1316,1386,11922,15799,0.0,NaN,NaN,1.0,1.0,0.0,0.0,45.0,0
7,2017-12-25,7,3,1220900,1316,1386,11922,15799,1.0,77.00,85.00,1.0,1.0,1.0,0.0,45.0,0
8,2017-12-24,8,3,1220900,1316,1386,11922,15799,1.0,77.00,85.00,1.0,1.0,1.0,0.0,46.0,0
9,2017-12-23,9,3,1220900,1316,1386,11922,15799,0.0,NaN,NaN,1.0,1.0,0.0,0.0,46.0,0


# 看

In [10]:
! ls tmp

0411_2018_brand			   13765_2018  ow_1315	     品类7054.csv
11922_2018			   data        test_debug
11922_cart_sjd_20180328151150.csv  datas       品类7052.csv


In [11]:
! ls tmp/data

0425_result	 870_p2.da			 simplified_ts_7052.csv
0720-data.ipynb  870_season.csv			 simplified_ts_7054.csv
7052_p1.da	 870_ts.da			 simplified_ts_870.csv
7052_p2.da	 area_rdc_mapping.csv		 test_0705
7052_ts.da	 holidays.csv			 to_xiaolu
7054_p1.da	 predict_p1_7052_2018_02_01.csv  train_p1_7052_2018_02_01.csv
7054_p2.da	 predict_p1_7052.csv		 train_p1_7052.csv
7054_ts.da	 predict_p1_7054_2018_02_01.csv  train_p1_7054_2018_02_01.csv
736_p1.da	 predict_p1_7054_2018_07_05.csv  train_p1_7054_2018_07_05.csv
736_p2.da	 predict_p1_7054.csv		 train_p1_7054.csv
736.tar.gz	 predict_p1_870_2018_02_01.csv	 train_p1_870_2018_02_01.csv
736_ts.da	 predict_p1_870.csv		 train_p1_870.csv
870_p1.da	 shishang


In [ ]:
to_del = ['areatypearray', 'productdesc', 'mainproductkey', 'bd_subtype1_giveaway', 'bd_subtype4_threshold1',
           'bd_subtype4_giveaway1', 'bd_subtype4_threshold2', 'bd_subtype4_giveaway2', 'bd_subtype4_threshold3',
           'bd_subtype4_giveaway3', 'bd_subtype2_threshold', 'bd_subtype2_giveaway', 'bd_subtype2_maximumgiveaway',
           'bd_subtype15_thresholdnumber1', 'bd_subtype15_giveawayrate1', 'bd_subtype15_thresholdnumber2', 'bd_subtype15_giveawayrate2',
           'bd_subtype15_thresholdnumber3', 'bd_subtype15_giveawayrate3', 'bd_subtype6_thresholdnumber', 'bd_subtype6_freenumber', 'fg_productkey',
           'dd_percent', 'bd_subtype1_percent', 'bd_subtype4_percent', 'bd_subtype2_percent', 'bd_subtype15_percent',
           'bd_subtype6_percent', 'suit_percent', 'fg_percent', 'allpercentdiscount']

f01处理前的表格 再f01 preprocessing 里面把空值比例超过20%的去掉（这个数字可以调整）

In [ ]:
! ls tmp/data/

In [5]:
p1_df = pd.read_csv('tmp/data/train_p1_7052_2018_02_01ws.csv')
p2_df = pd.read_csv('tmp/data/7052_p2.da',sep='\t',names=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
             'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
             'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
             'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
             'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt'])
ts_df = pd.read_csv('tmp/data/7052_ts.da',sep='\t',header=None,names=['Date', 'ind', 'RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag'])

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
## ts_df preprocessing ####
ForecastStartDate = pd.to_datetime(scenario['forecastStartDate'])
DataStartDate = ForecastStartDate - datetime.timedelta(days=scenario['lookbackPeriodDays'])
ts_df['Date'] = pd.to_datetime(ts_df['Date'])
ts_df_pred = ts_df[(ts_df['Date']<ForecastStartDate)&(ts_df['Date']>DataStartDate)]
ts_df_pred.replace('null', np.nan, inplace=True)
ts_df_pred.replace(-999, np.nan, inplace=True)
ts_to_float_col = ['RDCKey', 'ProductKey', 'HierarchyLevel1Key', 'HierarchyLevel2Key', 'HierarchyLevel3Key', 'brand_code', 'sales', 'priceAfterDiscount', 'jd_prc', 'vendibility', 'counterState', 'salesForecast', 'reserveState', 'stockQuantity', 'utc_flag']
if 'object' in ts_df_pred[ts_to_float_col].dtypes.values:
    object2Float(ts_df_pred,ts_to_float_col)

/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ubuntu/lijingjie/ljj/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
code/refactor/common.py:2178: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[col] = df[col].astype('float')


convert to Float columns:
RDCKey
convert to Float columns:
ProductKey
convert to Float columns:
HierarchyLevel1Key
convert to Float columns:
HierarchyLevel2Key
convert to Float columns:
HierarchyLevel3Key
convert to Float columns:
brand_code
convert to Float columns:
sales
convert to Float columns:
priceAfterDiscount
convert to Float columns:
jd_prc
convert to Float columns:
vendibility
convert to Float columns:
counterState
convert to Float columns:
salesForecast
convert to Float columns:
reserveState
convert to Float columns:
stockQuantity
convert to Float columns:
utc_flag


In [10]:
feature_select(ts_df_pred)

TypeError: float() argument must be a string or a number

In [12]:
ts_7054 = pd.read_csv('tmp/data/7052_ts.da',sep='\t')
ts_7054

,2018-04-24,0,3,1317085,1319,1523,7052,8241,0,NaN,NaN.1,0.0,0.0.1,0.1,0.0.2,0.0.3,1
0,2018-04-23,1,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
1,2018-04-22,2,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
2,2018-04-21,3,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
3,2018-04-20,4,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
4,2018-04-19,5,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
5,2018-04-18,6,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
6,2018-04-17,7,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
7,2018-04-16,8,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
8,2018-04-15,9,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1
9,2018-04-14,10,3,1317085,1319,1523,7052,8241,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1


In [4]:
p1_raw = pd.read_csv('tmp/data/7052_p1.da',sep='\t')
p2_raw = pd.read_csv('tmp/data/7052_p2.da',sep='\t')

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
p1_raw.columns=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
                        'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
                        'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
                        'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
                        'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
                        'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
                        'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
                        'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']